In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten, SimpleRNN, LSTM
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score

# 1. Chuẩn bị dữ liệu
df = pd.read_csv('student_scores.csv')
encoder = LabelEncoder()
df['Subject'] = encoder.fit_transform(df['Subject'])

# Phân loại hạng dựa trên điểm
def classify_score(score):
    return "Yếu" if score == 1 else "Trung bình" if score == 2 else "Khá" if score == 3 else "Giỏi"
df['Rank'] = df['Score'].apply(classify_score)

# Chuẩn bị dữ liệu đầu vào và nhãn cho dự đoán điểm và phân loại hạng
X = df[['Subject']].values
y_score = to_categorical(df['Score'] - 1)  # Dự đoán điểm
y_rank = to_categorical(df['Rank'].apply(lambda x: ["Yếu", "Trung bình", "Khá", "Giỏi"].index(x)))  # Dự đoán hạng

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra cho cả điểm và hạng
X_train, X_test, y_train_score, y_test_score = train_test_split(X, y_score, test_size=0.2, random_state=42)
_, _, y_train_rank, y_test_rank = train_test_split(X, y_rank, test_size=0.2, random_state=42)

# 2. Định nghĩa các mô hình
def build_model_7layer(input_shape, output_units):
    model = Sequential([
        Dense(100, activation='relu', input_shape=(input_shape,)),
        Dropout(0.5),
        Dense(100, activation='relu'),
        Dropout(0.5),
        Dense(100, activation='relu'),
        Dropout(0.5),
        Dense(100, activation='relu'),
        Dropout(0.5),
        Dense(output_units, activation='softmax')
    ])
    return model

def build_model_cnn(input_shape, output_units):
    model = Sequential([
        Conv1D(100, kernel_size=1, activation='relu', input_shape=(input_shape, 1)),
        MaxPooling1D(pool_size=1),
        Dropout(0.5),
        Conv1D(100, kernel_size=1, activation='relu'),
        Dropout(0.5),
        Flatten(),
        Dense(100, activation='relu'),
        Dropout(0.5),
        Dense(output_units, activation='softmax')
    ])
    return model

def build_model_rnn(input_shape, output_units):
    model = Sequential([
        SimpleRNN(100, activation='relu', input_shape=(input_shape, 1), return_sequences=True),
        Dropout(0.5),
        SimpleRNN(100, activation='relu', return_sequences=True),
        Dropout(0.5),
        SimpleRNN(100, activation='relu'),
        Dropout(0.5),
        Dense(output_units, activation='softmax')
    ])
    return model

def build_model_lstm(input_shape, output_units):
    model = Sequential([
        LSTM(100, activation='relu', input_shape=(input_shape, 1), return_sequences=True),
        Dropout(0.5),
        LSTM(100, activation='relu', return_sequences=True),
        Dropout(0.5),
        LSTM(100, activation='relu'),
        Dropout(0.5),
        Dense(output_units, activation='softmax')
    ])
    return model

# Khởi tạo các mô hình
models = {
    '7-layer_score': build_model_7layer(X_train.shape[1], y_train_score.shape[1]),
    '7-layer_rank': build_model_7layer(X_train.shape[1], y_train_rank.shape[1]),
    'cnn_score': build_model_cnn(X_train.shape[1], y_train_score.shape[1]),
    'cnn_rank': build_model_cnn(X_train.shape[1], y_train_rank.shape[1]),
    'rnn_score': build_model_rnn(X_train.shape[1], y_train_score.shape[1]),
    'rnn_rank': build_model_rnn(X_train.shape[1], y_train_rank.shape[1]),
    'lstm_score': build_model_lstm(X_train.shape[1], y_train_score.shape[1]),
    'lstm_rank': build_model_lstm(X_train.shape[1], y_train_rank.shape[1]),
}

# 3. Huấn luyện các mô hình
for model_name, model in models.items():
    print(f"Training model: {model_name}")
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    if 'score' in model_name:
        model.fit(X_train, y_train_score, epochs=50, batch_size=32, validation_data=(X_test, y_test_score))
    else:
        model.fit(X_train, y_train_rank, epochs=50, batch_size=32, validation_data=(X_test, y_test_rank))

# 4. Đánh giá các mô hình
def evaluate_model(model, X_test, y_test, model_name, task_type='Score'):
    y_pred = np.argmax(model.predict(X_test), axis=1)
    y_true = np.argmax(y_test, axis=1)
    if task_type == 'Score':
        mae = mean_absolute_error(y_true, y_pred)
        mse = mean_squared_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        print(f"{model_name} ({task_type}) - MAE: {mae}, MSE: {mse}, RMSE: {rmse}")
    else:
        accuracy = accuracy_score(y_true, y_pred)
        print(f"{model_name} ({task_type}) - Accuracy: {accuracy}")

# Đánh giá tất cả các mô hình
for model_name, model in models.items():
    if 'score' in model_name:
        evaluate_model(model, X_test, y_test_score, model_name, task_type='Score')
    else:
        evaluate_model(model, X_test, y_test_rank, model_name, task_type='Rank')

# 5. Dự đoán điểm và phân loại hạng theo Student_ID
def predict_student_by_id(student_id):
    student_data = df[df['Student_ID'] == student_id]
    if student_data.empty:
        print(f"Student ID {student_id} not found.")
        return
    
    subject = student_data['Subject'].values[0]
    subject_encoded = np.array([[subject]])

    # Dự đoán điểm và hạng
    score_prediction = np.argmax(models['7-layer_score'].predict(subject_encoded), axis=1)[0] + 1
    rank_prediction = np.argmax(models['7-layer_rank'].predict(subject_encoded), axis=1)[0]
    rank_label = ["Yếu", "Trung bình", "Khá", "Giỏi"][rank_prediction]

    print(f"Student ID: {student_id}")
    print(f"Predicted Score: {score_prediction}")
    print(f"Predicted Rank: {rank_label}")

# Ví dụ: Dự đoán điểm cho sinh viên với ID 'SV123'
predict_student_by_id('SV123')


c:\Users\ADMIN\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\ADMIN\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\ADMIN\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Training model: 7-layer_score
Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.4025 - loss: 1.3782 - val_accuracy: 0.5300 - val_loss: 1.2919
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5359 - loss: 1.2680 - val_accuracy: 0.5300 - val_loss: 1.2257
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5704 - loss: 1.2027 - val_accuracy: 0.5300 - val_loss: 1.2046
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5652 - loss: 1.1665 - val_accuracy: 0.5300 - val_loss: 1.2003
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5649 - loss: 1.1754 - val_accuracy: 0.5300 - val_loss: 1.1997
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5274 - loss: 1.2093 - val_accuracy: 0.5300 - val_loss: 1.1934
Epoch 7/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5456 - loss: 1.1596 - val_accuracy: 0.5300 - val_loss: 1.1921
Epoch 8/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5650 - loss: 1.1378 - v